<a href="https://colab.research.google.com/github/FatemaSamir/Head-Pose-Estimation/blob/main/Extract_featuers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import cv2
from google.colab.patches import cv2_imshow
import mediapipe as mp
import scipy.io as sio
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#unZiping
from zipfile import ZipFile
file_name = "/content/drive/MyDrive/Ml1_project/AFLW2000-3D.zip"
with ZipFile(file_name,'r') as ZIB:
 ZIB.extractall()
 print('Done')

Done


In [5]:
folder_path = '/content/AFLW2000/'
# files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))] ) # read only files imges and mat files
images_paths = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and ( f.split('.')[1] == 'jpg')])
mats_paths = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and ( f.split('.')[1] == 'mat')])

In [6]:
mats_paths[-1]

'image04375.mat'

In [50]:
# # Loading AFLW2000 Data
# %%capture
# if os.path.isfile('/content/drive/MyDrive/Ml1_project/AFLW2000-3D.zip') == False:
#   !gdown --id  1fP3zvSCYjll_o_m7S12nvQLZ9MnsEoap
#   !unzip /content/drive/MyDrive/Ml1_project/AFLW2000-3D.zip

# Extract Landmarks using mediapipe

In [8]:
folder_path = '/content/AFLW2000/'
# files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))] ) # read only files imges and mat files
images_paths = list(sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and ( f.split('.')[1] == 'jpg')]))
mats_paths = list(sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and ( f.split('.')[1] == 'mat')]))

# Create The DataFrame for featuears (land marks from face using media pip)

All_Data = pd.DataFrame(columns=['ID']+list(['F'+str(i) for i in range(1, (468*2)+1)]) + ['Yaw', 'Pitch', 'Roll'])

i = 0
faceModule = mp.solutions.face_mesh
for img_path, mat_path in zip(images_paths, mats_paths):
    L = []
    with faceModule.FaceMesh(static_image_mode=True) as face:
        # read 
        image = cv2.imread(folder_path + img_path)
         # processing the face to extract the landmark points (468 point) for each x,y,z
        results = face.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None:
            # Loop over faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    # save x and y for point landmark  
                    x = landmark.x
                    y = landmark.y
                    L.append(x)
                    L.append(y)

                # find 'Yaw', 'Pitch'and  'Roll'
                L = [img_path] + L + list(sio.loadmat(folder_path+mat_path)['Pose_Para'][0][:3])

                All_Data.loc[i] = L
                i += 1


In [9]:
All_Data.shape

(1853, 940)

In [10]:
All_Data.head(10)

,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F930,F931,F932,F933,F934,F935,F936,Yaw,Pitch,Roll
0,image00002.jpg,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,...,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,-0.399231,0.018227,0.085676
1,image00004.jpg,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,...,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,0.470065,1.189533,0.300959
2,image00006.jpg,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,...,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,-0.184650,0.881137,-0.236852
3,image00008.jpg,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,...,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,-0.175379,0.299208,-0.373374
4,image00013.jpg,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,...,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,-0.026812,0.011965,-0.220662
5,image00014.jpg,0.492562,0.659880,0.484423,0.589555,0.490822,0.613645,0.471540,0.528240,0.483233,...,0.491427,0.522413,0.492968,0.626868,0.476350,0.637145,0.471961,0.057119,0.110732,-0.043283
6,image00019.jpg,0.504923,0.660781,0.504300,0.590029,0.504618,0.615602,0.489647,0.534049,0.504232,...,0.505533,0.532564,0.506306,0.629079,0.496832,0.638516,0.492394,0.223228,0.022474,-0.006809
7,image00020.jpg,0.500070,0.677207,0.463701,0.625045,0.484325,0.635931,0.437842,0.569959,0.455318,...,0.503987,0.476720,0.510525,0.564460,0.454512,0.572303,0.445920,-0.170780,0.383899,-0.436852
8,image00021.jpg,0.499343,0.663182,0.500274,0.599654,0.500382,0.619543,0.488135,0.538709,0.500890,...,0.502106,0.535079,0.504181,0.636537,0.491866,0.645640,0.486357,-0.026928,-0.024356,-0.056860
9,image00022.jpg,0.432730,0.639559,0.409750,0.582831,0.440336,0.603227,0.428533,0.534794,0.408958,...,0.500745,0.491224,0.503503,0.573364,0.485283,0.583688,0.479568,0.236453,1.250053,0.167051


In [11]:
# save All Data 
All_Data.to_csv('/content/drive/MyDrive/Ml1_project/Notebookes/All_DataSet.csv',header=True)

In [12]:
All_Data.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F930,F931,F932,F933,F934,F935,F936,Yaw,Pitch,Roll
count,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,...,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000,1853.000000
mean,0.505036,0.666149,0.503698,0.611075,0.504510,0.627596,0.490213,0.550968,0.503416,0.592989,...,0.503258,0.530948,0.507431,0.621081,0.485716,0.629547,0.478221,-0.095366,0.013611,-0.033167
std,0.041261,0.028346,0.053271,0.029041,0.037840,0.024675,0.044587,0.023908,0.054824,0.028645,...,0.023010,0.033922,0.022426,0.047496,0.038984,0.050135,0.041212,0.549292,0.580266,0.649282
min,0.256906,0.179093,0.227159,0.144636,0.249772,0.152404,0.222207,0.100594,0.223030,0.131856,...,0.055864,0.266672,0.060640,0.304541,0.032022,0.294275,0.024076,-12.457202,-6.130066,-16.969362
25%,0.482141,0.652247,0.469521,0.598221,0.481937,0.617026,0.460069,0.540801,0.466724,0.580285,...,0.491350,0.514206,0.496217,0.593103,0.463229,0.601160,0.454632,-0.246453,-0.256714,-0.160274
50%,0.505162,0.668768,0.501536,0.614621,0.503587,0.629840,0.485371,0.552197,0.501211,0.596475,...,0.502106,0.532342,0.506317,0.626023,0.485355,0.635011,0.477745,-0.123371,0.019403,-0.017463
75%,0.526248,0.682794,0.537543,0.627792,0.525717,0.641242,0.516599,0.562998,0.538674,0.609476,...,0.513950,0.546662,0.517966,0.650143,0.506548,0.660787,0.499799,0.028133,0.305791,0.126460
max,0.928491,0.799190,0.929157,0.776498,0.923774,0.782066,0.901552,0.745973,0.926027,0.767816,...,0.720868,0.908686,0.723342,0.990572,0.741343,0.998658,0.750775,15.598820,3.277500,2.166107


In [21]:
# split ALL Data to  files YAW, PITCH and ROLL
YAW_DF = All_Data.drop(['Pitch', 'Roll'], axis = 1)
PITCH_DF = All_Data.drop(['Yaw','Roll'], axis = 1)
ROLL_DF = All_Data.drop(['Yaw','Pitch'], axis = 1)

In [22]:
print(YAW_DF.shape)
print(PITCH_DF.shape)
print(ROLL_DF.shape)

YAW_DF.to_csv('/content/drive/MyDrive/Ml1_project/Notebookes/YAW_DataSet.csv',header=True)
PITCH_DF.to_csv('/content/drive/MyDrive/Ml1_project/Notebookes/PITCH_DataSet.csv',header=True)
ROLL_DF.to_csv('/content/drive/MyDrive/Ml1_project/Notebookes/ROLL_DataSet.csv',header=True)

(1853, 938)
(1853, 938)
(1853, 938)


In [23]:
YAW_DF.head(5)

,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F928,F929,F930,F931,F932,F933,F934,F935,F936,Yaw
0,image00002.jpg,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,...,0.485137,0.548437,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,-0.399231
1,image00004.jpg,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,...,0.506956,0.511393,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,0.470065
2,image00006.jpg,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,...,0.717794,0.330334,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,-0.184650
3,image00008.jpg,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,...,0.500958,0.497731,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,-0.175379
4,image00013.jpg,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,...,0.481912,0.510743,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,-0.026812


In [24]:
PITCH_DF.head(5)

,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F928,F929,F930,F931,F932,F933,F934,F935,F936,Pitch
0,image00002.jpg,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,...,0.485137,0.548437,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,0.018227
1,image00004.jpg,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,...,0.506956,0.511393,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,1.189533
2,image00006.jpg,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,...,0.717794,0.330334,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,0.881137
3,image00008.jpg,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,...,0.500958,0.497731,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,0.299208
4,image00013.jpg,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,...,0.481912,0.510743,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,0.011965


In [25]:
ROLL_DF.head(5)

,ID,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F928,F929,F930,F931,F932,F933,F934,F935,F936,Roll
0,image00002.jpg,0.485651,0.686890,0.489194,0.639721,0.489802,0.647256,0.479730,0.562529,0.490558,...,0.485137,0.548437,0.492813,0.541429,0.500371,0.665668,0.471336,0.676091,0.464005,0.085676
1,image00004.jpg,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,...,0.506956,0.511393,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,0.300959
2,image00006.jpg,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,...,0.717794,0.330334,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798,-0.236852
3,image00008.jpg,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,...,0.500958,0.497731,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,-0.373374
4,image00013.jpg,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,...,0.481912,0.510743,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,-0.220662


In [29]:
# Split Data to Featuers and Labels
X = All_Data.drop(['ID','Yaw',	'Pitch',	'Roll' ], axis=1)
Y = All_Data[['Yaw',	'Pitch',	'Roll' ]]
print(X.shape)
print(Y.shape)

(1853, 936)
(1853, 3)


In [30]:
# split Data Train and Validation
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state =42)

In [32]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR,SVC
svr = MultiOutputRegressor(SVR()).fit(X_train, Y_train)
print('Training Score:\n', svr.score(X_val, Y_val))

Training Score:
 0.8284640121238439
